In [699]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request, response, parse
import re
import pickle
import html2text
import stop_words
from itertools import islice
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="white")
%matplotlib inline
path_file = '/Users/gowitheflow/Documents/ETALAB/algo_gouv/'

In [243]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

In [2]:
import random
import time
from IPython.display import clear_output

In [134]:
import secrets
secrets.token_hex(10)

'5687c3e487e63a41a821'

## Import google links

In [156]:
google_links = pd.read_csv('QUERY_RESULTS_google.csv', index_col=False).drop('Unnamed: 0', axis=1)
google_links.head(2)

,query,link_title,link,n_pages,type_of_doc,circulaire,affich,autre_type,type_affich,type_affichTexte,scraped,link_id
0,algorithmes%2Bautomatique+site:%22legifrance.g...,"Cour de cassation, civile, Chambre civile 1, 1...",https://www.legifrance.gouv.fr/affichJuriJudi....,11,affichJuriJudi.do?oldAction=rechJuriJudi&idTex...,0,1,0,affichJuriJudi,NaN,False,9fc4b03b89b16fb0
1,algorithmes%2Bautomatique+site:%22legifrance.g...,Arrêté du 17 octobre 2012 modifiant la méthode...,https://www.legifrance.gouv.fr/affichTexte.do?...,11,affichTexte.do?cidTexte=JORFTEXT000026601023&c...,0,1,0,affichTexte,Arrêté,False,01680d71f4c7703b


## Type de document ?

Commençons par distinguer les différents types de documents juridiques nous trouvons dans la base.

In [7]:
# On enlève les https et les http
links_clean_http = google_links.link.str.replace(
    'https://www.legifrance.gouv.fr/', '').str.replace(
    'http://www.legifrance.gouv.fr/', '').str.replace(
    'http://legifrance.gouv.fr/', '').str.replace(
    'https://legifrance.gouv.fr/', '')

google_links['type_of_doc'] = links_clean_http
# Détection des circulaires.
mask_circulaire = pd.Series([x.startswith('http') for x in links_clean_http])
mask_affich = pd.Series([x.startswith('affich') for x in links_clean_http])
google_links['circulaire'] = mask_circulaire.astype(int)
# Détection des affichTexte
google_links['affich'] = mask_affich.astype(int)
# Détection des autre type
google_links.loc[(google_links.circulaire == 0) & (google_links.affich == 0), 'autre_type'] = 1
google_links.autre_type.fillna(0, inplace=True)
google_links['autre_type'] = google_links.autre_type.astype(int)

google_links.loc[google_links.affich == 1, "type_affich"] = [x[0] for x in google_links.loc[google_links.affich == 1].type_of_doc.str.split('.do')]
google_links.loc[google_links.affich == 1, "type_affich"] = google_links.loc[google_links.affich == 1, "type_affich"].replace('affichTexteArticle', 'affichTexte')
# Type d'affichTexte
type_affichTexte = [x[0] for x in google_links[google_links.type_affich == 'affichTexte'].link_title.str.split(' ')]
google_links.loc[google_links.type_affich == 'affichTexte', 'type_affichTexte'] = type_affichTexte
# clean
google_links.type_affichTexte.replace('Arręté', 'Arrêté', inplace=True)


3 types de documents principaux :

1. Les circulaires 
-> (ex : 

2. Les documents commençant par "affich" :
    - **affichTexte**
        - Les arrêtés (ex:https://www.legifrance.gouv.fr/affichTexte.do?cidTexte=JORFTEXT000026601023&categorieLien=id)
        - Les avis (ex: https://www.legifrance.gouv.fr/affichTexte.do?cidTexte=JORFTEXT000033851257&categorieLien=id)
        - Les délibérations (ex : https://www.legifrance.gouv.fr/affichTexte.do?cidTexte=JORFTEXT000031346155)
        - Les décrets (ex: https://www.legifrance.gouv.fr/affichTexte.do?cidTexte=LEGITEXT000005631691&dateTexte=20041025)
        - ...
    - **affichJuriJuri**
        - Textes cours de cassation (ex:https://www.legifrance.gouv.fr/affichJuriJudi.do?oldAction=rechJuriJudi&idTexte=JURITEXT000026181998)
    - **affichCnil**
        - Délibération (ex: https://www.legifrance.gouv.fr/affichCnil.do?id=CNILTEXT000027824156)
    - **affichIDCC**
        - Convention collective (ex: https://www.legifrance.gouv.fr/affichIDCC.do;jsessionid=385F29B7DD9591CBA7F4E74F310D0C72.tpdjo16v_1?idSectionTA=KALISCTA000005728051&cidTexte=KALITEXT000005683657&idConvention=KALICONT000005635185)
        - Accord (il n'y en a qu'un) (ex: https://www.legifrance.gouv.fr/affichIDCC.do;jsessionid=385F29B7DD9591CBA7F4E74F310D0C72.tpdjo16v_1?idSectionTA=KALISCTA000005728051&cidTexte=KALITEXT000005683657&idConvention=KALICONT000005635185)
            - REMARQUE : les scraper semblent indentiques.
    - **affichAdmin**
        - Cours administrative d'Appel (CAA) (ex: https://www.legifrance.gouv.fr/affichJuriAdmin.do?idTexte=CETATEXT000028991191)
        - Conseil d'Etat (ex : https://www.legifrance.gouv.fr/affichJuriAdmin.do?idTexte=CETATEXT000027410895)
            - REMARQUE : les scraper semblent indentiques.
        
3. Les autres documents

In [686]:
google_links.type_affich.value_counts()

affichTexte             110
affichJuriJudi           20
affichCnil               10
affichJuriAdmin           9
affichIDCC                9
affichIDCCArticle         5
affichCode                2
affichCodeArticle         2
affichLoiPubliee          2
affichLoiPreparation      1
Name: type_affich, dtype: int64

# How not to be banned

- Switch user-agent
- Put sleeps between requests


In [10]:
_UA_list = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-GB; rv:1.8.1.6) Gecko/20070725 ',
    'Chrome/6.0.472.63 Safari/534.3',
    'Firefox/2.0.0.6',
    'Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.3 (KHTML, like Gecko)',
    'Chrome/6.0.472.63 Safari/534.3',
    'Safari/419.3',
    'Mozilla/5.0',
    'Mozilla/5.0 (Linux; U; Android 0.5; en-us) AppleWebKit/522+ (KHTML, like Gecko) ',
    'Opera/9.00 (Windows NT 5.1; U; en)',
    'Version/3.0 Mobile/1A543a Safari/419.3',
    'Mozilla/5.0 (iPhone; U; CPU like Mac OS X; en) AppleWebKit/420+ (KHTML, like Gecko)',
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:46.0) Gecko/20100101 Firefox/46.0',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:41.0) Gecko/20100101 Firefox/41.0'
]

_QUERIES = [
    'algorithmes%2Bautomatique+site:%22legifrance.gouv.fr%22',
    'classement%2Bautomatique+site:%22legifrance.gouv.fr%22',
    'cryptographie+site:legifrance.gouv.fr',
    'analyse+algorithme+-cryptographie+-cryptographiques+-cryptographique+%2Bsite:legifrance.gouv.fr',
    '%22affectation+automatique%22%2Bsite:legifrance.gouv.fr'
]

RESULT_DICT = {}

In [88]:
## Initialize variable "scraped" to know if a file as been scraped
google_links['scraped'] = False

## Scraping des `affichTexte` par type (`type_affichTexte`)

In [8]:
google_links.type_affichTexte.value_counts()

Arrêté          62
Avis            15
Délibération    12
Décret           9
Circulaire       3
LOI              3
Décision         2
Vocabulaire      2
Observations     1
Ordonnance       1
Name: type_affichTexte, dtype: int64

In [89]:
text_maker = html2text.HTML2Text()
text_maker.bypass_tables = True
text_maker.RE_MD_BACKSLASH_MATCHER = True
text_maker.ESCAPE_SNOB = True
text_maker.PROTECT_LINKS = True
text_maker.IGNORE_TABLES= True
text_maker.RE_UNESCAPE = True
text_maker.USE_AUTOMATIC_LINKS = True
text_maker.UNICODE_SNOB = True
text_maker.BODY_WIDTH = 0

In [391]:
def scrap_affichTexte(type_affichTexte):
    """ Scrap les différents type d'affichTexte.
    
    typeaffichTexte (str) : nom du type d'affichText (ex: "Arrêté")
    """
    
    google_links["scraped"] = _is_scraped(google_links.link_id)

    # Scrap mask
    mask_type = (google_links.type_affichTexte == type_affichTexte)
    mask_todo = (google_links.scraped == False)
    
    df = google_links.loc[mask_type & mask_todo, ["link_id", "link"]]
    print("%s: %s lien(s) restant(s) à scrapper !" % (type_affichTexte, len(df)))

    link_titles = []
    file_names = []
    html_text_list = []
    for row in df.iterrows():
        file_name = "%s_%s" % (type_affichTexte, row[1]["link_id"])
        # Request
        UA = _change_identity()
        header = {'User-Agent': UA} # Cause I'm not a bot!
        page = request.Request(row[1]["link"], headers=header)

        # Beautify results & convert to text
        html = request.urlopen(page)
        soup = BeautifulSoup(html, "lxml")
        html_as_text = text_maker.handle(soup.text)
        html_text_list.append(html_as_text)
        # Titre
        if (len(soup.findAll('div', {'class':'enteteTexte'})) > 0):
            link_titles.append(soup.findAll('div', {'class':'enteteTexte'})[0].find('strong').text)
        else:
            link_titles.append('aucun_titre')
        
        file_names.append(file_name)

        # Store text
        with open("text/affichTexte/%s.txt" % file_name, "w") as text_file:
            text_file.write(html_as_text)

    
    google_links.loc[mask_type & mask_todo, "link_title"] = link_titles
    google_links.loc[mask_type & mask_todo, "file_name"] = file_names
    google_links.loc[mask_type & mask_todo, "scraped"] = True
    google_links.loc[mask_type & mask_todo, "html_as_text"] = html_text_list


def _is_scraped(serie):
    """
    Check for each link_id (serie) if its been scraped already (by checking the generated files).
    
    Returns: a Serie of boolean.
    """
    already_scraped = [x.split('_')[1].split('.txt')[0] for x in os.listdir('./text/affichTexte/') if x.endswith('.txt') ]

    mask_scraped = [x in already_scraped for x in serie]

    return mask_scraped

#scrap_affichTexte("Arrêté")

In [320]:
scrap_affichTexte("Arrêté")

Arrêté: 62 lien(s) restant(s) à scrapper !


In [394]:
for type_aff in google_links.type_affichTexte.dropna().unique():
    scrap_affichTexte(type_aff)

Arrêté: 62 lien(s) restant(s) à scrapper !
Ordonnance: 1 lien(s) restant(s) à scrapper !
Avis: 15 lien(s) restant(s) à scrapper !
Délibération: 12 lien(s) restant(s) à scrapper !
Vocabulaire: 2 lien(s) restant(s) à scrapper !
LOI: 3 lien(s) restant(s) à scrapper !
Décision: 2 lien(s) restant(s) à scrapper !
Décret: 9 lien(s) restant(s) à scrapper !
Circulaire: 3 lien(s) restant(s) à scrapper !
Observations: 1 lien(s) restant(s) à scrapper !


In [722]:
google_links.to_csv('legifrance_links.csv', sep=';')